In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

используем ВСЕ поля
Amount, Value и TransactionStartTime скейлим с помощью RobustScaler
Остальные поля - MinMaxScaler

In [2]:
df_trn = pd.read_csv('./data/training_le.csv')
df_tst = pd.read_csv('./data/test_le.csv')
df_sbm = pd.read_csv('./data/sample_submission.csv')

In [3]:
df_trn_cut = df_trn.drop(columns=['FraudResult'])
df_tst_cut = df_tst

In [4]:
df_trn_cut['TransactionStartTime'] = pd.to_datetime(df_trn_cut['TransactionStartTime'])
df_tst_cut['TransactionStartTime'] = pd.to_datetime(df_tst_cut['TransactionStartTime'])

In [5]:
columns4MinMaxScaler = [
    'BatchId',
    'AccountId', 
    'SubscriptionId', 
    'CustomerId', 
    'ProviderId',
    'ProductId', 
    'ProductCategory', 
    'ChannelId',
    'PricingStrategy'
]

In [6]:
for clm in columns4MinMaxScaler:
    df_trn_cut[clm] = df_trn_cut[clm].astype(float)
    df_tst_cut[clm] = df_tst_cut[clm].astype(float)

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

In [8]:
def scaleColumns(data, cols_to_scale, scaler):
    for col in cols_to_scale:
        data[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data[col])))
    return data

In [9]:
df_trn_cut = scaleColumns(df_trn_cut, columns4MinMaxScaler, MinMaxScaler())
df_tst_cut = scaleColumns(df_tst_cut, columns4MinMaxScaler, MinMaxScaler())

In [10]:
df_trn_cut = scaleColumns(df_trn_cut, ['Amount',
                                       'Value',
                                       'TransactionStartTime'], RobustScaler())
df_tst_cut = scaleColumns(df_tst_cut, ['Amount',
                                       'Value',
                                       'TransactionStartTime'], RobustScaler())

In [11]:
df_trn_cut.head()

,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,0.258949,0.817355,0.183078,0.589061,1.0,0.346154,0.000000,0.50,0.000000,0.000000,-1.058573,0.666667
1,0.112122,1.000000,0.791684,0.589061,0.6,0.192308,0.222222,0.25,-0.357895,-0.207407,-1.058568,0.666667
2,0.386684,0.873554,0.045511,0.626103,1.0,0.000000,0.000000,0.50,-0.175439,-0.105820,-1.058171,0.666667
3,0.733818,0.133678,0.451593,0.131987,0.0,0.769231,1.000000,0.50,6.666667,4.402116,-1.057405,0.666667
4,0.277996,1.000000,0.791684,0.131987,0.6,0.192308,0.222222,0.25,-0.576842,-0.075344,-1.057382,0.666667


In [12]:
df_trn_cut.dtypes

BatchId                 float64
AccountId               float64
SubscriptionId          float64
CustomerId              float64
ProviderId              float64
ProductId               float64
ProductCategory         float64
ChannelId               float64
Amount                  float64
Value                   float64
TransactionStartTime    float64
PricingStrategy         float64
dtype: object

## Plot

In [208]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches

In [ ]:
# New_df is from the random undersample data (fewer instances)
X = new_df.drop('Class', axis=1)
y = new_df['Class']

In [ ]:
# T-SNE Implementation
t0 = time.time()
X_reduced_tsne = TSNE(n_components=2, random_state=42).fit_transform(X.values)
t1 = time.time()
print("T-SNE took {:.2} s".format(t1 - t0))

In [ ]:
# PCA Implementation
t0 = time.time()
X_reduced_pca = PCA(n_components=2, random_state=42).fit_transform(X.values)
t1 = time.time()
print("PCA took {:.2} s".format(t1 - t0))

In [ ]:
# TruncatedSVD
t0 = time.time()
X_reduced_svd = TruncatedSVD(n_components=2, algorithm='randomized', random_state=42).fit_transform(X.values)
t1 = time.time()
print("Truncated SVD took {:.2} s".format(t1 - t0))

In [15]:
num_transactions = df_trn.shape[0]
num_otliers = df_trn['FraudResult'].value_counts()[1]
num_inliers = df_trn['FraudResult'].value_counts()[0]
outliers_fraction = num_otliers / num_transactions
print('outliers fraction:', round(outliers_fraction, 3))

outliers fraction: 0.002


## One-Class SVM (OCSVM)

In [16]:
from sklearn import svm

In [82]:
OCSVM = svm.OneClassSVM(nu=outliers_fraction, kernel="rbf", gamma=0.001)
# nu=0.99*outliers_fraction+0.01

In [83]:
OCSVM.fit(df_trn_cut)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='rbf',
      max_iter=-1, nu=0.00201752001839811, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

In [84]:
FraudPred_OCSVM = OCSVM.predict(df_trn_cut)

In [85]:
FraudPred_OCSVM = [ 1 if x == -1 else 0 for x in FraudPred_OCSVM ]

In [86]:
unique, counts = np.unique(FraudPred_OCSVM, return_counts=True)
dict(zip(unique, counts))

{0: 95473, 1: 189}

In [87]:
unique, counts = np.unique(df_trn['FraudResult'], return_counts=True)
dict(zip(unique, counts))

{0: 95469, 1: 193}

In [88]:
f1_score(df_trn['FraudResult'], FraudPred_OCSVM)

0.544502617801047

## Isolation Forest (IF)

In [89]:
from sklearn.ensemble import IsolationForest

In [98]:
IF = IsolationForest(behaviour='new', contamination=outliers_fraction, random_state=2410)

In [99]:
IF.fit(df_trn_cut)

IsolationForest(behaviour='new', bootstrap=False,
        contamination=0.00201752001839811, max_features=1.0,
        max_samples='auto', n_estimators=100, n_jobs=None,
        random_state=2410, verbose=0)

In [100]:
FraudPred_IF = IF.predict(df_trn_cut)

In [101]:
FraudPred_IF_replace = [ 1 if x == -1 else 0 for x in FraudPred_IF ]

In [102]:
unique, counts = np.unique(FraudPred_IF_replace, return_counts=True)
dict(zip(unique, counts))

{0: 95469, 1: 193}

In [103]:
unique, counts = np.unique(df_trn['FraudResult'], return_counts=True)
dict(zip(unique, counts))

{0: 95469, 1: 193}

In [104]:
f1_score(df_trn['FraudResult'], FraudPred_IF_replace)

0.3730569948186529

## Fitting an Elliptic Envelope (EE)

In [105]:
from sklearn.covariance import EllipticEnvelope

In [149]:
EE = EllipticEnvelope(contamination=outliers_fraction*1.15)
# contamination=outliers_fraction*1.5

In [150]:
EE.fit(df_trn_cut);

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.712542096776851 > -106.865728666443857). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-82.130408791797663 > -106.681046778931375). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-81.127515446514025 > -106.780414663771509). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.131680275763351 > -107.918192771509013). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.704600829311147 > -107.275716170856640). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-52.288163854297551 > -105.341892530995437). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-52.683850026518648 > -106.988206136094391). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.691922283035986 > -106.115814684665622). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-52.508419793149962 > -107.423624571224522). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-79.315409314930150 > -106.314177796166817). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.439927821485462 > -106.298312151791279). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-78.862633368999738 > -110.272525417897995). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-86.940715517594100 > -110.162510619140477). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-56.593919400794448 > -108.175677426498012). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-54.365425459285042 > -109.466798489525502). You may want to try with a higher value of support_fraction (current v

...

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-52.082368387309081 > -169.576913901673038). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.917516893477796 > -169.271757120286566). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-114.066948203030222 > -143.239050504133189). You may want to try with a higher value of support_fraction (current 

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-53.943497797337898 > -136.613157134308210). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-85.480630051959167 > -115.140600368841092). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-86.077740854941140 > -116.172190538727477). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-113.242338485215939 > -115.018065125748933). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-55.219278068226508 > -174.094204380193673). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-113.938588302248291 > -114.039512245471911). You may want to try with a higher value of support_fraction (current

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-113.960772809786647 > -116.024631650665782). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-53.807469416615788 > -113.184929601968221). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-56.958056416718307 > -112.833537630797721). You may want to try with a higher value of support_fraction (current 

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-107.076800574026009 > -113.505216696200236). You may want to try with a higher value of support_fraction (current value: 0.500).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-55.337309020340200 > -165.074843766888932). You may want to try with a higher value of support_fraction (current value: 0.500).
  RuntimeWarning)


In [151]:
FraudPred_EE = EE.predict(df_trn_cut)

In [152]:
FraudPred_EE = [ 1 if x == -1 else 0 for x in FraudPred_EE ]

In [153]:
unique, counts = np.unique(FraudPred_EE, return_counts=True)
dict(zip(unique, counts))

{0: 95440, 1: 222}

In [154]:
unique, counts = np.unique(df_trn['FraudResult'], return_counts=True)
dict(zip(unique, counts))

{0: 95469, 1: 193}

In [155]:
f1_score(df_trn['FraudResult'], FraudPred_EE)

0.7951807228915662

**submission**

In [158]:
df_sbm = pd.read_csv('./data/sample_submission.csv')

In [157]:
FraudPred_EE = EE.fit_predict(df_tst_cut)

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-46.148615307365382 > -106.055064458346436). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.111449884593974 > -106.558158787364022). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-47.068334643796717 > -105.872422669632527). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-47.398494078715444 > -106.857913830972706). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-47.118401338759526 > -106.877326496190719). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-47.598764681733144 > -106.930877611593473). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-47.696029636151160 > -106.097848714293121). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.881599171083572 > -106.562425552310714). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.245963101791979 > -106.713303565937366). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.181717102650097 > -109.132377167584963). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.218434377931459 > -106.332797550042912). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.213603114155738 > -107.147112347451042). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.810445012698835 > -108.365402434772491). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-80.502163812251126 > -108.035401438495171). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.848527710089321 > -108.569857104831982). You may want to try with a higher value of support_fraction (current v

...

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-81.767148034763864 > -108.438370077505766). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-81.799529263193548 > -139.781469739655591). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-54.055038292424094 > -138.431034861297036). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-84.182147718187636 > -111.688905224873196). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-84.953366379079910 > -111.816125938869590). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-82.312923686996612 > -138.423831734749456). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-53.061331526128491 > -136.314579727936746). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-82.978735379985039 > -138.361277336655235). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.372719963481501 > -135.445426325161947). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.747968078886480 > -140.037254890290797). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.848370241537225 > -137.553310611326310). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.607016395616256 > -135.887291784872701). You may want to try with a higher value of support_fraction (current v

/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-83.525209895150766 > -108.841909675956543). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-56.050695211562356 > -111.232495612374436). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/home/abogatyrov/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-55.456278670579302 > -110.483531171099386). You may want to try with a higher value of support_fraction (current v

In [161]:
FraudPred_EE = [ 1 if x == -1 else 0 for x in FraudPred_EE ]

In [162]:
df_sbm['FraudResult'] = FraudPred_EE

In [163]:
df_sbm.to_csv('AlBo_07_08.csv', encoding='utf-8', index=False)

## Local Outlier Factor (LOF)

In [164]:
from sklearn.neighbors import LocalOutlierFactor

In [202]:
LOF = LocalOutlierFactor(n_neighbors=12, contamination=outliers_fraction)

In [203]:
FraudPred_LOF = LOF.fit_predict(df_trn_cut)

In [204]:
FraudPred_LOF = [ 1 if x == -1 else 0 for x in FraudPred_LOF ]

In [205]:
unique, counts = np.unique(FraudPred_LOF, return_counts=True)
dict(zip(unique, counts))

{0: 95469, 1: 193}

In [206]:
unique, counts = np.unique(df_trn['FraudResult'], return_counts=True)
dict(zip(unique, counts))

{0: 95469, 1: 193}

In [207]:
f1_score(df_trn['FraudResult'], FraudPred_LOF)

0.11917098445595854